# <font class='ign_color'>TF in ROS</font>

## Chapter 3: Robot State Publisher

You already know the basics for TF publication and subscription.<br>
But as you might have already noticed, it would be really tedious and difficult to publish the TF of articulated robots, such as robot arms and humanoids. Not to mention the fact that you would need to know the exact joint configurations of each moving and fixed element of the robot. Not practical at all.<br>

<table style="width:100%">
  <tr>
    <th>
    <figure>
      <img id="fig-A.1" src="img/robonaut_notf.png" width="300"/>
       <center> <figcaption><h2>Robonaut2</h2></figcaption></center>
    </figure>

    </th>
    <th>
        <figure>
      <img id="fig-A.2" src="img/robonaut_onlytf.png" width="300"/>
       <center> <figcaption><h2>Robonaut2 Only TF</h2></figcaption></center>
    </figure>
    </th>
    
    <th>
        <figure>
      <img id="fig-A.2" src="img/robonaut_tf_nonames.png" width="300"/>
       <center> <figcaption><h2>Robonaut2 with TF</h2></figcaption></center>
    </figure>
    </th> 

  </tr>
</table>

So, what can you do?<br>
Well, ROS gives you a nice tool called <b>robot state publisher</b>.<br>
In essence, it takes a file describing the morphology of the robot as input and it publishes the TF.<br>
This is a great leap forward because now you can work with really complex TF structures.<br>
And that's what you are going to do right now.

# 1. Know how Pi-Robot works

You are going to work with a simpler robot than the Robonaut2 for practical reasons. The robot is called Pi-Robot.<br>
Despite being much simpler, if you learn how to work with Pi-Robot, you will be able to understand many other more complex robots. You will only have to learn the particular way in which each robot file system has been arranged.

<figure>
  <img id="fig-A.1" src="img/pi_robot.png" width="300"/>
   <center> <figcaption><h2>Fully articulated Pi-Robot</h2></figcaption></center>
</figure>

The best way to see the structure is to see how the TF frames are arranged. So, let's get started!

<p style="background:#EE9023;color:white;">Exercise 3.1</p>
<br>
In this exercise, you are going to launch a pre-made rosbot_state_publisher launch with control and you are going to see the TF structure with tools you already know.<br>

0.- The first step is to activate the control and robot state publisher.
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch pi_robot_pkg pi_robot_control.launch

You should see that the joints of the robot become rigid and stop moving around freely.<br><br>
1.- Move the Robot. For that, you will have to publish a command to one of the available joints. You can have a list of all the available joints by executing a rostopic list<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic list | grep command

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic pub /pi_robot/head_tilt_joint_position_controller/command std_msgs/Float64 "data: 0.0"                
rostopic pub -1 /pi_robot/head_pan_joint_position_controller/command std_msgs/Float64 "data: 0.7"

You should get movements like this one:

<img src="img/pi_robot_moving.gif" width=150/>

These oscillations are due to the PID controller values used, the weight, and the torque applied.

2.- Use the TF processing tools explained in Unit1 to have a better idea of how the Pi-Robot is structured:<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>


In [ ]:
rosrun tf view_frames
rosrun rqt_tf_tree rqt_tf_tree
rostopic echo -n1 /tf
rosrun tf tf_echo /frame_you_choose_1 /frame_you_choose_2
rosrun rviz rviz

<p style="background:#EE9023;color:white;">END Exercise 3.1</p>

You should have had a similar output as the following:

[view_frames pi_robot_full_tf](extra_files/pi_robot_full_tf.pdf)

[rqt_tf_tree pi_robot_full_tf](extra_files/pi_robot_rqt_tf_tree.svg)

<img src="img/rviz_pirobot1.png"/>

In [ ]:
user ~ $ rosrun tf tf_echo torso_link upper_base_link                                                                           
At time 637.077                                                                                                                 
- Translation: [0.000, 0.000, -0.100]                                                                                           
- Rotation: in Quaternion [0.000, 0.000, -0.000, 1.000]                                                                         
            in RPY (radian) [0.000, 0.000, -0.000]                                                                              
            in RPY (degree) [0.000, 0.000, -0.000]

## 2. Create your own robot_state_publisher launch

<p style="background:#EE9023;color:white;">Exercise 3.2</p>
<br>
In this exercise, you will create your own robot_state_publisher launch and see what happens when executing it by itself.<br>

0.- The first step is to STOP the control and robot state publisher that you launched in Exercise 3.1.<br>
**<i>roslaunch pi_robot_pkg pi_robot_control.launch</i>**<br>

1.- The next step is to create your own simple launch file. It should be something like this:

In [2]:
<launch>
    
  <!-- Load the URDF file in the param server variable robot_description if it wasn't loaded before -->
  <param name="robot_description" command="cat $(find pi_robot_pkg)/urdf/pi_robot_v2.urdf" />

  <!-- Publish TF with robot_state_publisher -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"
    respawn="false" output="screen">
    <remap from="/joint_states" to="/pi_robot/joint_states" />
  </node>

</launch>

SyntaxError: invalid syntax (<ipython-input-2-64cb3db7ec7e>, line 1)

Here you save the **pi_robot_v2.urdf** robot definition inside a ROS parameter named **robot_description**. From here, the robot_state_publisher will take all the information that it needs.<br>

In [3]:
<!-- Load the URDF file in the param server variable robot_description if it wasn't loaded before -->
  <param name="robot_description" command="cat $(find pi_robot_pkg)/urdf/pi_robot_v2.urdf" />

SyntaxError: invalid syntax (<ipython-input-3-ff90d748d354>, line 1)

The second part is launching the robot_state_publisher and remapping the **/joint_states** topic. Remapping is not really needed, but it's done for better namespacing of the simulation.<br>
It's here that the robot_state_publisher will start to publish the TF of all the links defined in the URDF file.

In [ ]:
<!-- Publish TF with robot_state_publisher -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"
    respawn="false" output="screen">
    <remap from="/joint_states" to="/pi_robot/joint_states" />
  </node>

2.- You are now ready to launch your_robot_state_publisher.launch. Once done, take a look at the rqt_tf_tree, or the view_frames, or the RVIZ. Is something odd happening?<br>

<p style="background:#EE9023;color:white;">END Exercise 3.2</p>

You should have a tf_tree similar to this one:

[rqt_tf_tree pi_robot_only_statepublisher](extra_files/pi_robot_no_jointstates.svg)

As you can see in the tf_tree, the links are not connected. You just see some of them connected. Why?<br>
Let's have a sneak peek into the URDF file that defines the Pi-Robot.<br>
For that, you will copy the URDF file to your catkin_ws/src to be able to see it nicely in the IDE:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p>
</th>
</tr>
</table>
<br>

In [ ]:
roscp pi_robot_pkg pi_robot_v2.urdf /home/user/catkin_ws/src/

### Observations of pi_robot_v2.urdf
You might be asking yourself: Why do I have to see how the robot is defined? What does this have to do with TFs?<br>
Well, actually a lot. How you define the robot model is how the robot state publisher will publish its TFs.
<ul>
<li>This URDF was written in a way in which everything was in one single file. In bigger projects, the robot definition is divided into several files to make it more modular and easy to maintain. Although, this simple way allows you to see all of the Pi-Robot in sequence</li>.
<li>The URDF file is mainly divided into three parts: Links Definition, Joints Definition, and Materials/Colors Definition. </li>
</ul>

<ul>
    <li>Let's have a closer look at the joint definitions and the two types: Fixed and Non-Fixed.</li>
</ul>

This is a <b>Fixed</b> joint that connects the torso_link to the head_pan_link.<br>

In [ ]:
<joint name="head_pan_servo" type="fixed">
    <parent link="torso_link"/>
    <child link="head_pan_link"/>
    <origin xyz="0 0 0.225" rpy="0 0 0"/>
</joint>

This is a <b>Non-Fixed</b> joint, continuous in this case. It connects the **<i>head_pan_link</i>** with the **<i>head_tilt_link</i>**. Note that in the [rqt_tf_tree pi_robot_only_statepublisher](extra_files/pi_robot_no_jointstates.svg), the **<i>head_pan_link</i>** appears, but not the **<i>head_tilt_link</i>**.<br>
This is because while the **<i>head_pan_link</i>** is in some fixed joints, the **<i>head_tilt_link</i>** is only connected to other links through Non-Fixed joints.<br>

In [ ]:
<joint name="head_pan_joint" type="continuous">
        <parent link="head_pan_link"/>
        <child link="head_tilt_link"/>
        <origin xyz="0 0 0.045" rpy="0 0 0"/>
        <axis xyz="0 0 1"/>
        <dynamics damping="0.7"/>
    </joint>

Why are only fixed joint links represented like frames in the TF?

The answer is: **Because the robot state publisher knows how to calculate their TFs, while it doesn't know how to do so with the Non-Fixed joints.**

This is because Non-Fixed joints, like this **<i>head_pan_joint</i>**, can be in any configuration, radically changing the Tranform or TF. It depends on the value of the joint, like it would with servo actuated joints.<br>

How do we solve this? Simple, you just need to publish the value of the Non-Fixed joint, how much the servo has turned, and the reading of the encoder.<br>
And here is where the <b>joint_state_publisher</b> comes in.

## 3. Joint State Publisher

To solve the Non-Fixed joints value issue that prevents us from having a fully connected TF tree, you just have to publish the joint state value.<br>
This can be done one of two main ways:<br>
<ol>
    <li>Publish them directly.</li>
    <li>Make the hardware publish the joint_states through the readings of encoders or similar systems.</li>
</ol>

### 3.1. Publish the joint_states directly

<p style="background:#EE9023;color:white;">Exercise 3.3</p>
<br>
In this exercise, you will publish the joint_states of all the movable joints through a GUI and see them move in RVIZ.<br>

0.- The first step is to STOP the robot state publisher that you launched in Exercise 3.2.<br>

1.- The next step is to create your own new launch file. This time, it should run the robot_state_publisher and also publish the joint_state values.<br>
Here is an example of a cleaner version of a robot_state_publisher and a joint_state_publisher through a GUI:

In [4]:
<launch>
  <param name="robot_description" command="cat $(find pi_robot_pkg)/urdf/pi_robot_v2.urdf" />

  <!-- send fake joint values -->
  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="use_gui" value="TRUE"/>
  </node>

  <!-- Combine joint values -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="state_publisher"/>

  <!-- Show in RVIZ   -->
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find pi_robot_pkg)/launch/pi_robot.rviz"/>

</launch>

SyntaxError: invalid syntax (<ipython-input-4-a44701eb9d52>, line 1)

Let's comment on each part:

In [ ]:
<param name="robot_description" command="cat $(find pi_robot_pkg)/urdf/pi_robot_v2.urdf" />

<!-- Combine joint values -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="state_publisher"/>

This part is exactly the same as in exercise 3.2. It just doesn't have a remap.

In [ ]:
<!-- send fake joint values -->
  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="use_gui" value="TRUE"/>
  </node>

This part runs the joint_state_publisher and opens a GUI with which you will be able to change the joint values with sliders and based on the joint limits, if there are any.

In [5]:
<!-- Show in RVIZ   -->
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find pi_robot_pkg)/launch/pi_robot.rviz"/>

SyntaxError: invalid syntax (<ipython-input-5-03ad13162cf9>, line 1)

Here you are launching RVIZ, loading an RVIZ configuration file to have it all correctly set up. This last step is not necessary, but it makes things faster.

2.- Launch your new launch file and open the graphical tools. Locally, it would be exactly the same, but it would pop up on your screen.

<img src="img/font-awesome_desktop.png"/>

You should get something similar to this:

<img src="img/rviz_jointstate.gif"/>

<img src="img/pi_robot_jointstatepublisher.png"/>

Now, move the sliders around. You should see the joints moving around in RVIZ, with their TF frames. This means that now the robot state publisher knows how to calculate the TFs.

You can execute the rqt_tf_tree now and see that it has a complete TF tree.

<p style="background:#EE9023;color:white;">END Exercise 3.3</p>

But wait... what about the robot in the simulation?<br>
It's not moving. In fact, maybe the arms are still wandering around freely. Why?<br>
Although the joint state values are published, and therefore, the TFs can be calculated, these are just internal variables of ROS. Nothing here is telling the arms to move.<br>
It would be exactly the same on a real robot.<br>
If you had a way to publish the encoder values into your system and change them, it wouldn't mean the real robot would move.<br>
And here is where the MovingControllers come into play.

### 3.2 Joint State Publish through hardware readings 

Obviously, the best way to have the joint state values is through readings in the hardware. In the case of Gazebo simulations, you have to use a <b>controller_manager</b> ROS package.<br>
It will allow you to simulate different types of PID control, like Position Control or Effort Control.<br>
These are the steps to set up a URDF model to control joints:

#### Step 1: Define a joint that isn't fixed in your URDF

For example, take this from the pi_robot_v2.urdf:

In [ ]:
<joint name="left_shoulder_forward_joint" type="revolute">
    <parent link="left_shoulder_link"/>
    <child link="left_shoulder_forward_link"/>
    <origin xyz="0 0.025 0" rpy="0 0 0"/>
    <limit lower="-1.57" upper="1.57" effort="10" velocity="3"/>
    <axis xyz="0 0 1"/>
    <dynamics damping="0.7"/>
</joint>

#### Step 2: Define a transmission with the name of the joint in URDF

For example, take this from the pi_robot_v2.urdf:

In [ ]:
<transmission name="tran4">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="left_shoulder_forward_joint">
        <hardwareInterface>EffortJointInterface</hardwareInterface>
    </joint>
    <actuator name="motor4">
        <hardwareInterface>EffortJointInterface</hardwareInterface>
        <mechanicalReduction>1</mechanicalReduction>
    </actuator>
</transmission>

<font style="color:green">**NOTE:** If the joint is defined as revolute, but the limits are 3.14 and -3.14, then change them to continuous. Otherwise, it won't work.</font>

#### Step 3: Define the new transmission controller with the name xxx__position_controller (left_shoulder_forward_joint_position_controller) in a configuration yaml file:

For example, take this from the pirobot_control.yaml:

In [ ]:
pi_robot:
  # Publish all joint states -----------------------------------
  joint_state_controller:
    type: joint_state_controller/JointStateController
    publish_rate: 50
   
  left_shoulder_forward_joint_position_controller:
    type: effort_controllers/JointPositionController
    joint: left_shoulder_forward_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}

You always have to add the robot model name at the start; in this case, pi_robot to a namespace, and the joint_state_controller with its parameters, like publish_rate.

#### Step 4: Start the controller through the launch file

An example from the pi_robot_control.launch:

In [ ]:
<!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find pi_robot_pkg)/config/pirobot_control.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
    output="screen" ns="/pi_robot" args="x1_controller x2_controller left_shoulder_forward_joint_position_controller joint_state_controller"/>

#### Errors that might appear in the process:

1) Could not switch controllers, due to resource conflict: This is when you assigned the same joint to different controllers.

#### Use rqt_gui to see the state of the controllers:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #X</p>
</th>
</tr>
</table>


In [ ]:
rosrun rqt_gui rqt_gui

From the 'Plugins' menu, add the 'Topics->Message Publisher' and add, for example, the:

**<i>/pi_robot/left_elbow_joint_position_controller/command</i>**

Set Publish rate to 100Hz and click **ADD**.

Then, open the data field and write a valid Python Experion from math or random modules in it:

**<i>cos(i/100)</i>**

Finally, check the box to activate it. This will publish a new value each second, making the elbow oscillate forwards and backwards.

<img src="img/rqt_gui.png"/>

<img src="img/pi_robot_sinusmovement.gif"/>

### 3.3 See joint_state publishing up close

These two methods do exactly the same thing: publish into a topic named /joint_states.<br>
If you execute the echo command, you should get something similar to this. Be careful with the data overflow because it's a lot of data published very fast:

In [ ]:
rostopic echo /pi_robot/joint_states

header:                                                                                                                         
  seq: 122823                                                                                                                   
  stamp:                                                                                                                        
    secs: 2496                                                                                                                  
    nsecs: 922000000                                                                                                            
  frame_id: ''                                                                                                                  
name: ['head_pan_joint', 'head_tilt_joint', 'left_elbow_joint', 'left_shoulder_forward_joint', 'left_shoulder_up_joint', 'left_w
rist_joint', 'right_elbow_joint', 'right_shoulder_forward_joint', 'right_shoulder_up_joint', 'right_wrist_joint', 'torso_joint']
position: [1.0761352381649658e-05, 0.6000000203268394, 0.6854240516242394, -0.0027175423501786256, 1.2723288486718847, 0.0057816
84509972074, -0.2927324891222973, 0.002874571950627569, 1.2774441293401972, -0.0057604623506710695, 0.000338311049429052]       
velocity: [0.00036306626519703674, -5.087971091906217e-06, 0.9839388100687485, 0.0004649343343647767, 0.05150733303477194, -0.00
020205744075042992, -9.730541789929378e-07, 0.00047472216379803147, -2.505188206409192e-06, -1.754343438191663e-05, 0.0007127576
179940816]                                                                                                                      
effort: [-0.005015719090106785, 0.0, -1.9983986636700202, -0.0168876458182865, 1.8182624779766954, 0.0013147581146455423, 0.5306
305757115659, -0.0069206886654349375, 1.8038611002543448, -0.0012777518214157624, -0.0415055498225847]

You get the name of the joint, its position, and the effort it endures.<br>

<a href="http://wiki.ros.org/controller_manager">More Info on ROS controller_manager package</a><br>
<a href="http://wiki.ros.org/joint_state_publisher">More Info on ROS joint_state_publisher package</a><br>

<p style="background:#EE9023;color:white;">Exercise 3.4</p>
<br>
In this exercise, you will publish the joint_states of all the movable joints by activating the joint controllers already defined in the pi_robot_v2.urdf.<br>

0.- The first step is to STOP the robot state publisher and joint_state_publishers that you launched in prior exercises.<br>

1.- The next step is to create your own new launch file. This time, it should run the robot_state_publisher and run the joincontrollers.<br>
Here is an example:

In [ ]:
<launch>
    
    
  <param name="robot_description" command="cat $(find pi_robot_pkg)/urdf/pi_robot_v2.urdf" />

  <!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find pi_robot_pkg)/config/pirobot_control.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
    output="screen" ns="/pi_robot" args="head_pan_joint_position_controller head_tilt_joint_position_controller torso_joint_position_controller left_shoulder_forward_joint_position_controller right_shoulder_forward_joint_position_controller left_shoulder_up_joint_position_controller right_shoulder_up_joint_position_controller left_elbow_joint_position_controller right_elbow_joint_position_controller left_wrist_joint_position_controller right_wrist_joint_position_controller joint_state_controller"/>

  <!-- convert joint states to TF transforms for rviz, etc -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"
    respawn="false" output="screen">
    <remap from="/joint_states" to="/pi_robot/joint_states" />
  </node>

</launch>

You already know most of the code, so just comment on the controller part:

In [ ]:
<!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find pi_robot_pkg)/config/pirobot_control.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
    output="screen" ns="/pi_robot" args="head_pan_joint_position_controller head_tilt_joint_position_controller torso_joint_position_controller left_shoulder_forward_joint_position_controller right_shoulder_forward_joint_position_controller left_shoulder_up_joint_position_controller right_shoulder_up_joint_position_controller left_elbow_joint_position_controller right_elbow_joint_position_controller left_wrist_joint_position_controller right_wrist_joint_position_controller joint_state_controller"/>

Here you will load the controller configuration yaml and start the controllers through the controller_manager spawner system.<br>
You just have to specify which controllers defined in the yaml you want to activated.

3.- Once done, check that the tf_tree is now complete and you can visualise all the TFs moving at the same time that the real robot moves.

<img src="img/pi_robot_final.gif"/>

<p style="background:#EE9023;color:white;">END Exercise 3.4</p>

## Congratulations!

You now know how to publish TFs manually and through the robot_state publisher.<br>
You also have learned valuable skills to actuate your own robot and move it around.